# UMAMI Demo

This notebook demonstrates how to generate an UMAMI plot from a .csv file that has been generated by the `summarize_job.py` script included with pytokio.

In [ ]:
import os
import json
import pandas
import datetime
import tokio.analysis.umami

We _must_ include `parse_dates` for the column which will be passed as each `UmamiMetric`'s timestamp.

In [ ]:
df = pandas.read_csv('sample_summary.csv.gz',
                     parse_dates=['_datetime_start', '_datetime_end'],
                     date_parser=lambda x: datetime.datetime.fromtimestamp(float(x))).set_index('index')

## Filter data

We can select jobs that only match certain criteria by performing boolean operators on the DataFrame to generate filters.  We can then combine these filters using the `&` operator.

In this example, we apply the following filters:

1. Select only jobs that did more writes than reads.  The `darshan_biggest_{read,write}_api_bytes` keys contain the total number of bytes read or written by any single I/O API (POSIX or MPI-IO).

2. Select only jobs that performed most of their I/O to the file system mounted at `/scratch2`.

In [ ]:
### Select only jobs of a certain motif
filtered_indices = [x.endswith('hacc_io_write') for x in df['darshan_app']]

### Select only jobs that did more writes than reads (this is actually redundant since hacc_io_write only does writes)
filtered_indices &= df['darshan_biggest_write_api_bytes'] > df['darshan_biggest_read_api_bytes']

### Select jobs in a specific time range
filtered_indices &= df['_datetime_start'] >= datetime.datetime(2017, 12, 7)
filtered_indices &= df['_datetime_start'] < datetime.datetime(2017, 12, 21)

### and also select only jobs whose I/O went to a file system mounted at a specific place.
### Note that we have to check the type because Pandas will load empty CSV values as NaN,
### which cannot be .startswith()ed.
filtered_indices &= [ type(x) == str and x.startswith('/global/cscratch') for x in df['darshan_biggest_write_fs'] ]


### Filter out dates where Darshan says we wrote more than Lustre thinks
filtered_indices &= df['darshan_total_gibs_posix'] <= df['fs_tot_gibs_written']

We can now view a few example rows from our filtered data.  Note that we use `.T` to transpose the example rows just so you can see all of the metrics contained in this DataFrame.

In [ ]:
df[filtered_indices].head().T

## Build the Umami object

We want to look at the following metrics:

1. `darshan_agg_perf_by_slowest_posix`, converted to GiB/s
2. `darshan_total_gibs_posix` / ( `lmt_tot_gibs_read` + `lmt_tot_gibs_written` )
3. `fshealth_ost_most_full_pct`
4. `lmt_ave_mds_cpu`
5. `lmt_max_oss_cpu`

For each one, we create an `UmamiMetric` object, then we build the `Umami` object from them.  `UmamiMetric` objects can be given anything list-like (i.e., can be sliced in exactly one dimension), or `pandas.Series` objects.  In this example, we are passing it `pandas.Series` objects that are pulled straight out of our `df` DataFrame.

Note that the following code is expanded out to make it easy to read; in practice, it's more concise to iteratively create and add new `UmamiMetric`s to the parent `Umami` object.

In [ ]:
metric1 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['darshan_agg_perf_by_slowest_posix'] / 1024.0,
    label="Performance (GiB/sec)",
    big_is_good=True)

metric2 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['darshan_total_gibs_posix'] /
        (df[filtered_indices]['fs_tot_gibs_read'] + df[filtered_indices]['fs_tot_gibs_written']),
    label="Coverage Factor (Bandwidth)",
    big_is_good=True)

metric3 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['fs_max_oss_cpu'],
    label="Maximum OSS CPU Load (%)",
    big_is_good=False)

metric4 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['fs_ave_mds_cpu'],
    label="Average MDS CPU Load (%)",
    big_is_good=False)

metric5 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['fs_tot_open_ops'] / 1.0e6,
    label="Total open ops (MOps)",
    big_is_good=False)

metric6 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['fshealth_ost_most_full_pct'],
    label="Most Full OST (% Used)",
    big_is_good=False)

metric7 = tokio.analysis.umami.UmamiMetric(
    timestamps=df[filtered_indices]['_datetime_start'],
    values=df[filtered_indices]['topology_job_max_radius'],
    label="Maximum Job Radius",
    big_is_good=False)

Construct an `Umami` object with the five metrics we defined above.

In [ ]:
umami = tokio.analysis.umami.Umami()
umami['performance'] = metric1
umami['cf_bw'] = metric2
umami['max_oss_cpu'] = metric3
umami['ave_mds_cpu'] = metric4
umami['tot_opens'] = metric5
umami['fullest_ost'] = metric6
umami['radius'] = metric7

## Examine the UMAMI

The most obvious thing to do with an UMAMI is plot it.

In [ ]:
axes = umami.plot()
axes[0].get_figure().savefig('umami.pdf', bbox_inches='tight', transparent=True)

Its data can also be presented as a DataFrame.  Note that the `UmamiMetric` metadata (the label and whether bigger values are better or not) are lost when we do this.

In [ ]:
umami.to_dataframe()

`Umami` and `UmamiMetric` objects also serialize reasonably well.  The parent object can be converted either to a dictionary or a json string.

In [ ]:
print umami.to_dict()

In [ ]:
print umami.to_json()